# Scrape Naysh Y'All

Goal: by the end of this tutorial, you should have all the tools and skills you need to do some "simple" web scraping (i.e. static `GET` and `POST` requests only)

# Basic web-scraping concepts

This is by no means an exhaustive summary. there are 7.2 bajillion articles about web scraping and they're probably all better. [Especially the Mozilla ones](https://developer.mozilla.org/en-US/). Also, people always freak out about how bad the W3 pages are (and they are), but they're still (in my opinion) a good beginner's resource. So dont' be afraid to try them out, but be aware that there are some problems.

The only thing that is special about this one is that I wrote it and you've been duped into reading it.

So enjoy!

## requests

The basic problem here is that some one out there in the ol' interwebs has put up a pretty page with some data we want, and we need to go get it. The primary action we will be executing is called a `request`. We are trying to make our computer yell at their computer and send us that thing we like. So we need to figure out how to yell in a way that makes their computer happy.

Our computers all yell at eachother in a language called HTTP -- you have perhaps heard of it.

[There are a ton of details about how to yell](https://developer.mozilla.org/en-US/docs/Web/HTTP), what's appropriate and what isn't, etc. But for our purpose, we basically only care about [`requests`](https://developer.mozilla.org/en-US/docs/Web/HTTP/Methods). There are several different types of yells:

+ `GET` -- you want to get a thing and they should give it to you
+ `POST` -- you want to send them a thing and they should at least write you a thank you card; if they want to give you something back that's cool but it's not expected (but it's polite, so, that's basically the same as expectation, but with all sorts of passive aggressiveness tacked on)
+ `HEAD` -- you want to get a thing but you would prefer it if they just told you about the thing first, like sent you a picture of it or told you how big it was. You may still want to get it at some point but you're just testing the water
+ `PUT` -- a really pushy `POST` request, basically they tell us they have a cool painting and we say OUR PAINTING HERE IS BETTER EFF YOURS
+ `DELETE` -- actually that painting sucked, our bad
+ `CONNECT`, `OPTIONS`, `TRACE`, `PATCH` -- stuff your parents yell to their friends but no one really does any more outside of really formal settings

Of these, you basically only need `GET` and `POST`. Like, I had to use a `PUT` once and it was really annoying.

## data

So, suppose you got your computer to yell at their computer and everyone's into it. You say HEY CAN A FELLA `GET` OR WHAT and the other computer said

01011001 01100101 11101010 01011100 01001010 00001001 01101101 10111001 
00011010 00011010 00101100 00000101 01111111 10110000 01001001 10100111 
01010011 01101000 10010010 11110101 00011001 00001000 10100011 01000010 
11000111 00110011 00111110 00001000 11110010 10010101 00100101 01101101 
10010100 10011011 10111001 01100011 01110101 01111110 00100011 01010111 
10101111 10000111 10000101 01100101 00000000 11111110 10110011 11011110 
01011110 00100101 01011001 11101110 01101100 01000001 01010000 01101101 
01010011 11100010 01111010 10101101 11100011 00000001 01011010 01001010 
01000110 01011000 01011110 00000010 00110111 11111101 11001101 00000001 
11001101 01111001 10111001 10001010 11100010 11011011 01010100 10101101 
11101000 00000011 11110010 00001101 10100001 00101110 11000010 00110010 
01001000 11010001 10111000 11101011 01111000 00000100 10000110 10101010 
00110001 01111110 00101011 01110010 

not cool

### `html`

okay so a little cool, it looks like they speak [HyperText Markup Language](https://developer.mozilla.org/en-US/docs/Web/HTML). So at least they're trying. It's a special amalgam of xml-like tag structures that is the backbone of pretty much every single webpage.

check out [this example page with simple `html`](http://www.columbia.edu/~fdc/sample.html). Specifically, right click and go to "view source." You can do that on any webpage.

Of course, you usually will have more luck using the "developers tools" than just viewing source -- in google chrome, try right click > "Inspect"

#### `xpath`

[`xpath`](https://msdn.microsoft.com/en-us/library/ms256086(v=vs.110%29.aspx) is a query language for `xml` documents. Given that `html` is effectively a refined subset of `xml`, it's a natural fit for parsing `html` documents. It's a bit more time-tested and battle-worn than `css`, though not really as popular and (in my opinion) not as easy to learn. It's kind of the `C` or `C++` to `css`'s `python`.

that being said, it's by far my favorite of the two, and almost always my go-to. Just because I learned it first... and the edge cases are not as strange as the `css` edge cases, in my opinion. That being said, I'd probably recommend *learning* `css` selecting instead.

The best way to think of it, in my opinion, is like selecting files on a file system -- you know that somewhere several levels deep inside your root directory there is a folder called `important_files` and that it has a file in it where the extension is `.doc`. You could look for it (using wildcards in the linux world) by typing:

```bash
find . -name important_files/*.doc
```

you know that somewhere there's an element that is a child of something and has a property.

`xpath` is similar -- you know things about the hierarchy of a document (it's a list item `ul`) and the attributes (it is in the class `zachs_list`), and the `xpath` translation is

```html
<div>
    <ul class="zachs_list">
        <li>hello</li>
        <li>world</li>
    </ul>
</div>
```

```xpath
.//ul[@class="zachs_list"]
```

#### `css`

Cascading Style Sheets (*aka* [`css`](https://developer.mozilla.org/en-US/docs/Web/CSS)) is a language for describing how a browser should render `html` documents. they are not a pre-requisite for creating a webpage (styling can be done directly in `html` docs), but websites that don't utilize this language (or some more advanced variant) are few and far between.

If you want an example of `css`, you can again refer to the "Inspect" screen / "developers tools." 

+ The entire right panel is dedicated to `css` and `javascript` properties of the highlighted `html` document elements.
+ the `Sources` tab of the Inspect window has a lot of files, but all the delivered `css` files are there
+ check out the `css` file for [the Elder Research landing page](http://www.elderresearch.com/): [source is here](http://www.elderresearch.com/hs-fs/hub/2176909/hub_generated/template_assets/1491831211917/custom/page/css/vast-style.min.css), but that's not that useful, is it?  

So basically, there is a well-established language for telling browsers how to style "things like this" -- we can just use the existence of this language and statements such as those to take an `html` document and grab the same sorts of things.

```css
ul.zachs_list
```

### `json`

`html` isn't the only way that computers yell at each other, though. There are other languages they yell in, and one of the most common is [JavaScript Object Notation](http://json.org/), or `json` for short.

If you're familiar with `python` default data types, `json` can be converted directly to lists, dictionaries, numbers, and strings. My personal favorite example is [the Magic the Gathering `json` webpage](https://mtgjson.com/) ([direct link to example](https://mtgjson.com/json/KLD.json)), but [there are a ton out there](https://github.com/toddmotto/public-apis).

There are a million web tutorials out there. Use any of them. They're all better than this.

`json` is everywhere. it [has it's detractors](https://codepunk.io/xml-vs-json-why-json-sucks/), but it is essentially the *de facto* data and information transfer protocol on the web.

### javascript

A quick note related to `json` -- as I wrote above, `json` is the JavaScript Object Notation. The reason it's so ubiquitous is that [`javascript`](https://developer.mozilla.org/en-US/docs/Web/JavaScript) is ubiquitous. Javascript is a programming language which has a ton of bugs and features resulting from it's use case as the primary web language for client-side (aka heavy lifting is done by you (via your browser) rather than the server sending you information) computation.

Some people treat `javascript` as if it's not a chaos land of abominations and for the most part we all let them go on thinking those things, because they make us really pretty and fast webpages. Sometimes [everyone realizes that the whole enterprise is castles built on sand](https://qz.com/646467/how-one-programmer-broke-the-internet-by-deleting-a-tiny-piece-of-code/) and the super l33t trolls of the internet have a chuckle or two.

The main reason I bring it up is this: sometimes the only way to access the data you want is via `javascript`. In about 90% of all cases, I find this is something you can avoid with some digging. Why avoid it? because it requires breaking out of the simple `requests` paradigm and using a full-fledged `javascript` engine, and that's a big jump in complexity.

**Whenever possible, I try and just `GET` things.**

# setup

## command line tools

In the unix world, there are two primary utilities for web scraping:

+ `curl`
+ `wget`

Usually I default to `curl` for absolutely no reason.

### linux

```bash
sudo apt-get install curl wget
```

### mac

```bash
brew install curl wget
```

### windows

https://curl.haxx.se/download.html

god help you.

## `python`

First of all, you should have done most of the steps in `basic python - 001 - installing the basic data science toolkit`. If you haven't done that yet, go check out that mini-course.

I'm going to assume that you have a working installation of `conda` or `miniconda`. Let's create a new environment and install the most basic items for our first web scraping tests.

```bash
> conda create -n scrapesville python=3
> source activate scrapesville
> conda install requests lxml cssslect
```

that's it! we should be good to go

# simple example

Let's do some scraping!

## `GET` request

as a reminder, a `GET` request is a request we send to a server to ask for some information, and the returned message is whatever content we asked for.

### `json` is almost too easy

My go to `json` resource, as discussed above, is [mtgjson.com](https://mtgjson.com/). However, I can understand why maybe that's not the most useful resource for everyone. Let's work with the (debatably) more useful [wikipedia `json` api](https://www.mediawiki.org/wiki/API:Main_page)

https://en.wikipedia.org/w/api.php?action=query&titles=Leet&prop=revisions&rvprop=content&format=json

Let's break the url from this request down into pieces

+ `https://en.wikipedia.org/w/api.php` - the endpoint (this is a common term used for an api (application programming interface) url that provides us with some resource). One way of thinking of this is that it is the "function" name for this particular very generic function 
+ `?` - a common delimiter (most common?) indicating that what follows is a list of key-value pairs of parameters. This is like the open-parentheses of a function declaration
+ `foo=bar` - the "key = value" parameter list. the allowed keys are defined by the api endpoint, and a really good api will [tell you what the possibilities are](https://en.wikipedia.org/w/api.php) (but most won't).
+ `&` - the delimiter separating key-value pairs, like commas in an `R` or `python` function

altogether this makes for a an easier reading of the above url:

+ to to the primary wikipedia api endpoint, and
+ take the `action` of [`query`](https://en.wikipedia.org/w/api.php?action=help&modules=query)
+ look for documents with `titles` of `Leet`
+ look for the `prop`(erty) [`revisions`](https://en.wikipedia.org/w/api.php?action=help&modules=query%2Brevisions)
+ for each revision, get the property (aka the `rvprop`) `content`
+ return it to us with the `format` `json` (other options are `jsonfm` and `xml` -- in your browser, try changing that value in the url!

Let's do it with `python`!

In [ ]:
import requests

In [ ]:
requests.get?

In [ ]:
def my_request(title='Leet'):
    return requests.get(
        url='https://en.wikipedia.org/w/api.php',
        params={
            'action': 'query',
            'titles': title,
            'prop': 'revisions',
            'rvprop': 'content',
            'format': 'json'
        }
    )

In [ ]:
response = my_request(title='ham')
response.json()

We can check which [status code](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status) the server returned us. These are standardized codes that summarize the response we get from the server (e.g. "this was easy", "wtf are you talking about?!")

This is useful information for more automated process, but also sometimes helps us understand what went wrong. For the most part, we like 200s and we hate 400s and 500s. The others, supposedly, exist.

In [ ]:
response.status_code

awesome. Now what about the content?

In [ ]:
j = response.json()
j

In [ ]:
j['query']['pages']['18562']['revisions'][0]['*']

not too bad!

### a quick diversion: installing selector gadget

Not hard to explain -- go find the browser extension [selector gadget](http://selectorgadget.com/) and install it.

So let's suppose now that we found a webpage we want to scrape, but they haven't built an awesome `json` api for us to use. Let's use [the list of the most downloaded `python` packages](http://pypi-ranking.info/alltime), and let's try and get the package name and the numbers from this list.

Quick demo of selector gadget!

### `xpath` selection

let's just dive right in and discuss things as we go through the example

In [ ]:
import lxml.html
import requests

In [ ]:
response = requests.get(url='http://pypi-ranking.info/alltime')
response.status_code

In [ ]:
print(response.text)

first, we need to create an `lxml` `html` object out of the request response

In [ ]:
root = lxml.html.fromstring(response.text)
root

In [ ]:
root.getchildren()

let's look up the elements by hard-coding the path:

In [ ]:
root.xpath('body')

In [ ]:
root.xpath('body/div/div/table/tbody/tr/td/a/span')

what??

it turns out that browsers add the `tbody` element in there -- it's not in the code. Why is still mostly a mystery to me, and [I'm comforted by the fact that I'm not alone](https://stackoverflow.com/questions/27918086/why-tbody-will-be-added-automatically-by-browser).

In [ ]:
root.xpath('body/div/div/table/tr/td/a/span')

SO USEFUL RIGHT? Let's look at one of those items above

In [ ]:
packageElems = root.xpath('body/div/div/table/tr/td/a/span')
print('we have {} elements'.format(len(packageElems)))

elem0 = packageElems[0]
print(lxml.html.tostring(elem0))

In [ ]:
elem0.attrib

In [ ]:
elem0.text

so we can get all of the package names:

In [ ]:
packages = []
for elem in packageElems:
    packages.append(elem.text)

In [ ]:
packages = [elem.text for elem in packageElems]
packages[:5]

But that path -- it was pretty ugly, right? Is there a better way?

Well, it turns out that the elements we were looking for have a specific property:

```html
<span class="list_title">simplejson</span>
```

we could just pull all the spans which have that class, instead

In [ ]:
packageElems = root.xpath('.//span[@class="list_title"]')
packages = [elem.text for elem in packageElems]
packages[:5]

not bad!

and now for the number of downloads... Unforunately, the actual element we wish to pull doesn't have any special attributes -- but its parent (`td` element) does

In [ ]:
dlElems = root.xpath('.//td[@class="count"]/span')
elem0 = dlElems[0]
elem0

In [ ]:
elem0.text

In [ ]:
int(elem0.text)

In [ ]:
int(elem0.text.replace(',', ''))

In [ ]:
dlElems = root.xpath('.//td[@class="count"]/span')
dlnums = [int(elem.text.replace(',', '')) for elem in dlElems]
dlnums[:5]

In [ ]:
l1 = ['a', 'b', 'c']
l2 = [1, 2, 3]

dict(zip(l1, l2))

In [ ]:
list(zip(packages, dlnums))[:5]

In [ ]:
d = dict(zip(packages, dlnums))
d

In [ ]:
table = root.find('.//table[@id="main_list"]')
x = []
for row in table.xpath('./tr'):
    packagename = row.find('.//span[@class="list_title"]').text
    dlct = row.find('./td[@class="count"]/span').text
    dlct = int(dlct.replace(',', ''))
    x.append({'name': packagename, 'dlct': dlct})

In [ ]:
x

### `css` selection

as you saw above, we were generally referring to the class or `id` elements of `html` tag elements -- well, that's almost identical to what is done in `css`. Let's use selector gadget to select elements using `css` selectors.

Note: we will reuse the `lxml.html` object from before

In [ ]:
packageElems = root.cssselect('.list_title')
packages = [elem.text for elem in packageElems]
packages[:5]

In [ ]:
dlElems = root.cssselect('.count span')
dlnums = [int(elem.text.replace(',', '')) for elem in dlElems]
dlnums[:5]

In [ ]:
d = dict(zip(packages, dlnums))
d

# advanced example: `POST` form request

The most common use case for `POST` requests is `html` forms. There are forms all over the internet, and many have a `method` of `POST`. If you're looking for an example of a `POST` request, check out the github gist submission api:

https://developer.github.com/v3/gists/#create-a-gist

# `javascript` engines and `selenium`

sometimes a simple request is not enough. A good example is `cnn.com`, where one must scroll through the page to load new stories.

In [ ]:
import requests

resp = requests.post(
    url='https://www.eversource.com/clp/outage/services/OutageData.asmx/AllOutageData', 
    headers={'content-type': 'application/json'}
)

resp.json()